In [2]:
import pandas as pd
import numpy as np
import re
import os
from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

In [179]:
source = 'source/WPP2015_POP_F07_1_POPULATION_BY_AGE_BOTH_SEXES.XLS'

In [188]:
data_est = pd.read_excel(source, sheetname='ESTIMATES', skiprows=16, header=None, na_values='…')

In [189]:
data_var = pd.read_excel(source, sheetname='MEDIUM VARIANT', skiprows=16, header=None, na_values='…')

In [182]:
data_var.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,Index,Variant,"Major area, region, country or area *",Notes,Country code,Reference date (as of 1 July),0-4,5-9,10-14,15-19,...,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100+
1,1,Medium variant,WORLD,NaN,900,2015,670928,637449,607431,590069,...,339691,292727,215047,153207,114652,71449.7,37062.2,13388.5,2921.85,450.895
2,2,Medium variant,WORLD,NaN,900,2020,677600,664283,634568,604323,...,387339,320927,267875,187442,123356,81851.5,42701,16937.7,4242.6,605.465
3,3,Medium variant,WORLD,NaN,900,2025,673175,671930,661684,631509,...,428166,367326,295012,234798,152990,89822.1,49848,20116.4,5538.92,905.182
4,4,Medium variant,WORLD,NaN,900,2030,672121,667926,669744,658943,...,450092,407564,339529,260426,193066,113476,56221.9,24041.6,6836.46,1244.85


In [183]:
data_est.tail()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
3370,3370,Estimates,Tonga,NaN,776,1995,13.384,12.022,12.477,10.337,...,2.492,2.137,1.406,0.869,…,0.46,0.194,0.066,0.013,0.001
3371,3371,Estimates,Tonga,NaN,776,2000,13.28,12.928,11.307,11.539,...,2.473,2.087,1.672,0.991,…,0.539,0.231,0.071,0.016,0.002
3372,3372,Estimates,Tonga,NaN,776,2005,13.755,12.738,11.877,9.931,...,2.285,2.091,1.698,1.26,…,0.656,0.282,0.088,0.018,0.002
3373,3373,Estimates,Tonga,NaN,776,2010,13.748,13.05,12.1,10.896,...,2.22,1.896,1.627,1.255,…,0.837,0.351,0.112,0.023,0.003
3374,3374,Estimates,Tonga,NaN,776,2015,12.91,13.304,12.818,11.488,...,2.424,1.942,1.571,1.244,…,0.845,0.456,0.144,0.03,0.004


In [190]:
data_est.iloc[0, 6:] = data_est.iloc[0, 6:].map(lambda x: 'Total Population aged '+ x + ' (Number)')

In [191]:
data_est.columns = data_est.ix[0].map(to_concept_id)

In [192]:
data_est = data_est.drop(['index', 'notes'], axis=1)

In [193]:
data_est.head()

,variant,major_area_region_country_or_area,country_code,reference_date_as_of_1_july,total_population_aged_0_4_number,total_population_aged_5_9_number,total_population_aged_10_14_number,total_population_aged_15_19_number,total_population_aged_20_24_number,total_population_aged_25_29_number,...,total_population_aged_60_64_number,total_population_aged_65_69_number,total_population_aged_70_74_number,total_population_aged_75_79_number,total_population_aged_80_number,total_population_aged_80_84_number,total_population_aged_85_89_number,total_population_aged_90_94_number,total_population_aged_95_99_number,total_population_aged_100_number
0,Variant,"Major area, region, country or area *",Country code,Reference date (as of 1 July),Total Population aged 0-4 (Number),Total Population aged 5-9 (Number),Total Population aged 10-14 (Number),Total Population aged 15-19 (Number),Total Population aged 20-24 (Number),Total Population aged 25-29 (Number),...,Total Population aged 60-64 (Number),Total Population aged 65-69 (Number),Total Population aged 70-74 (Number),Total Population aged 75-79 (Number),Total Population aged 80+ (Number),Total Population aged 80-84 (Number),Total Population aged 85-89 (Number),Total Population aged 90-94 (Number),Total Population aged 95-99 (Number),Total Population aged 100+ (Number)
1,Estimates,WORLD,900,1950,337432,269550,260286,238628,221781,194424,...,73422.4,55106.4,37360.2,21997.3,14201.8,NaN,NaN,NaN,NaN,NaN
2,Estimates,WORLD,900,1955,402845,315055,263266,254815,231892,214878,...,76843.2,59321.8,40345.9,23755.2,16158.3,NaN,NaN,NaN,NaN,NaN
3,Estimates,WORLD,900,1960,430565,380319,309276,257899,248413,225957,...,85064.3,62665.1,44018.2,25985.8,18061.2,NaN,NaN,NaN,NaN,NaN
4,Estimates,WORLD,900,1965,477798,409020,372817,303891,251897,242692,...,95085.4,70255.6,47382.4,29457.4,21032,NaN,NaN,NaN,NaN,NaN


In [194]:
data_var.iloc[0, 6:] = data_var.iloc[0, 6:].map(lambda x: 'Total Population aged '+ x + ' (Number)')

data_var.columns = data_var.ix[0].map(to_concept_id)
data_var = data_var.drop(['index', 'notes'], axis=1)

In [195]:
concs = data_est.ix[0].T

In [196]:
concs.index.name = 'concept'
concs.name = 'name'

In [197]:
concs = concs.rename(index={'reference_date_as_of_1_july': 'year'})

In [198]:
concs = concs.reset_index()

In [199]:
concs['concept_type'] = 'string'

concs['concept_type'].iloc[2] = 'entity_domain'
concs['concept_type'].iloc[3] = 'time'
concs['concept_type'].iloc[4:] = 'measure'

In [200]:
concs['unit'] = np.nan

concs['unit'].iloc[4:] = 'thousands'

/usr/local/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [201]:
concs = concs.append(pd.DataFrame([
            ['unit', 'Unit', 'string', np.nan], 
            ['name', 'Name', 'string', np.nan]], columns=concs.columns))

In [202]:
concs.sort_index().head(20)

,concept,name,concept_type,unit
0,variant,Variant,string,NaN
0,unit,Unit,string,NaN
1,name,Name,string,NaN
1,major_area_region_country_or_area,"Major area, region, country or area *",string,NaN
2,country_code,Country code,entity_domain,NaN
3,year,Reference date (as of 1 July),time,NaN
4,total_population_aged_0_4_number,Total Population aged 0-4 (Number),measure,thousands
5,total_population_aged_5_9_number,Total Population aged 5-9 (Number),measure,thousands
6,total_population_aged_10_14_number,Total Population aged 10-14 (Number),measure,thousands
7,total_population_aged_15_19_number,Total Population aged 15-19 (Number),measure,thousands


In [203]:
concs.sort_index().to_csv('../ddf--concepts.csv', index=False)

In [77]:
# country

In [204]:
entity = data_est.drop(0)[['major_area_region_country_or_area', 'country_code']]

In [205]:
entity.drop_duplicates().to_csv('../ddf--entities--country.csv', index=False)

In [83]:
# data point

In [206]:
dp_1 = data_est.drop(0).drop('major_area_region_country_or_area', axis=1)
dp_2 = data_var.drop(0).drop('major_area_region_country_or_area', axis=1)

In [207]:
dp = pd.concat([dp_1, dp_2])

In [208]:
dp = dp.rename(columns={'reference_date_as_of_1_july': 'year'})

In [209]:
dp = dp.set_index(['country_code', 'year', 'variant'])

In [211]:
for name, df in dp.iteritems():
    path = os.path.join('../', 'ddf--datapoints--{}--by--country_code--year.csv'.format(name))
    df.reset_index().sort_values(by=['country_code', 'year']).dropna().to_csv(path, index=False)

In [168]:
from ddf_utils import ddf_reader as ddf

In [169]:
ddf.SEARCH_PATH = '/Users/semio/src/work/Gapminder/'

In [177]:
ddf.ddf_datapoint('ddf--gapminder--systema_globalis', 'crude_birth_rate_births_per_1000_population')

,geo,crude_birth_rate_births_per_1000_population,time
0,afg,48.14,1800
1,afg,48.14,1801
2,afg,48.14,1802
3,afg,48.14,1803
4,afg,48.14,1804
5,afg,48.14,1805
6,afg,48.14,1806
7,afg,48.14,1807
8,afg,48.14,1808
9,afg,48.14,1809
